#  Duplex Thermodynamics Calculator
###  Author: Yi Zeng
###  Last Updated: 2025-03-18 13:55 (CDT)
###  Notebook: calculate_duplex_thermodynamics.ipynb


This document provides a comprehensive Python implementation for calculating thermodynamic parameters (**ΔH**, **ΔS**, **ΔG**) of nucleic acid duplexes.  
It includes the precise handling of:
- **Perfectly matched duplexes** (DNA, RNA, and RNA-DNA hybrids)
- **Internal and terminal mismatches duplexes** (DNA only)
- **Dangling-end scenarios duplexes** (DNA and RNA)

The calculations rely on well-established **nearest-neighbor models** and experimentally derived **thermodynamic parameters** under **1 M NaCl solution**, with other salt concentrations varying based on reference data.

---

## **Key Implementation**

- **Direct Thermodynamic Calculation**: Computes **ΔH, ΔS, and ΔG** for duplexes, including perfect matches, mismatches, and dangling ends.
- **Added `DNA_NN5` Table**: Supports an additional nearest-neighbor model for improved accuracy.
- **Automated Shift in Dangling Duplexes**: Automatically aligns sequences for dangling-end corrections, requiring no manual shift input.

---

### **How to Use the Code**

#### **1. Run the Entire Script**
Run **all parts sequentially** from **Part 0 to Part 5** for proper execution.

#### **2. Execution Order**
- **Part 0:** Import packages and define thermodynamic parameters.
- **Part 1:** Set up helper functions.
- **Part 2:** Perform thermodynamic calculations.
- **Part 3:** Handle user input and selections.
- **Part 4:** Process calculations and results.
- **Part 5:** Execute the main function.

#### **3. Input Requirements**
- **Sequence (`seq`)**: For RNA/DNA hybridizations, `seq` must be the **RNA sequence**.
- **RNA Sequences**: Before performing calculations, **convert all `U` to `T`** to match thermodynamic parameter tables. This should be done for the first input when providing two sequences.
- **Temperature (`T`)**: Input temperature should be in **Kelvin (K)**.
- **Matching Duplex Calculation**: Requires **only one input strand**; the function automatically generates the complementary sequence.
- **Unmatching Duplex Calculation**: Requires **two input strands**:
  - **First strand (5' → 3')**: The main sequence.
  - **Second strand (3' → 5')**: The complementary sequence (input manually, not auto-generated).


#### **4. Run the Final Execution**
After running all parts, execute **Part 5** to call the main function:
```python
#  Run the calculator
if __name__ == "__main__":
    run_thermodynamics_calculator()
```
This will launch the interactive duplex thermodynamics calculator.

---

## **Further Information**
For more details, refer to the [Biopython MeltingTemp package](https://github.com/biopython/biopython/blob/master/Bio/SeqUtils/MeltingTemp.py).




# 0: Environment Setup & Global Parameters

Import required packages (Bio.SeqUtils.MeltingTemp, Bio.Seq, pandas, etc.).

Define thermodynamic parameter tables (nearest-neighbor (NN), dangling ends (DE), mismatches).

Add new table DNA_NN5.


In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.7 MB/s eta 0:00:00


In [ ]:
from Bio.SeqUtils.MeltingTemp import (
    #  DNA/DNA Hybridization Nearest-Neighbor Tables
    DNA_NN1,  # Breslauer et al. (1986) - Proc Natl Acad Sci USA 83: 3746-3750
    DNA_NN2,  # Sugimoto et al. (1996) - Nucleic Acids Res 24: 4501-4505
    DNA_NN3,  # Allawi & SantaLucia (1997) - Biochemistry 36: 10581-10594 (Default Biopython)
    DNA_NN4,  # SantaLucia & Hicks (2004) - Annu Rev Biophys Biomol Struct 33: 415-440

    #  RNA/RNA Hybridization Nearest-Neighbor Tables
    RNA_NN1,  # Freier et al. (1986) - Proc Natl Acad Sci USA 83: 9373-9377
    RNA_NN2,  # Xia et al. (1998) - Biochemistry 37: 14719-14735
    RNA_NN3,  # Chen et al. (2012) - Biochemistry 51: 3508-3522

    #  RNA/DNA Hybridization Nearest-Neighbor Table
    R_DNA_NN1,  # Sugimoto et al. (1995) - Biochemistry 34: 11211-11216

    #  Mismatch & Dangling Ends Corrections
    DNA_IMM1,  # Internal mismatches - Allawi & SantaLucia (1997-1998) Biochemistry 36-37
    DNA_TMM1,  # Terminal mismatches - SantaLucia & Peyret (2001) Patent WO 01/94611
    DNA_DE1,   # Dangling ends (DNA) - Bommarito et al. (2000) Nucleic Acids Res 28: 1929-1934
    RNA_DE1    # Dangling ends (RNA) - Turner & Mathews (2010) Nucleic Acids Res 38: D280-D282
)

from Bio.Seq import Seq
import pandas as pd
from IPython.display import display

In [ ]:
# SantaLucia et al. (1996), Biochemistry 35, 3555–3562
DNA_NN5 = {
    "init": (0, 0), "init_A/T": (0, 0), "init_G/C": (0, 0),
    "init_oneG/C": (0, -5.9), "init_allA/T": (0, -9.0), "init_5T/A": (0.4, 0),
    "sym": (0, -1.4),
    "AA/TT": (-8.4, -23.6), "AT/TA": (-6.5, -18.8), "TA/AT": (-6.3, -18.5),
    "CA/GT": (-7.4, -19.3), "GT/CA": (-8.6, -23.0), "CT/GA": (-6.1, -16.1),
    "GA/CT": (-7.7, -20.3), "CG/GC": (-10.1, -25.5), "GC/CG": (-11.1, -28.4),
    "GG/CC": (-6.7, -15.6)}

# 5′-terminal T`A bp
# To account for end effects, duplexes are given the penalty listed for
# each terminal 5′-T‚A-3′ base pair. Note this penalty is not applied to
# sequences with terminal 5′-A‚T-3′ base pairs (see text).

# 1: Initialization & Utilities

In [ ]:
def initialize_thermodynamics(nn_table, seq, c_seq=None):
    # If no complementary sequence provided, create it
    if not c_seq:
        c_seq = str(Seq(seq).complement())
    d_h, d_s = 0, 1

    # Initialize ΔH and ΔS from the selected nn_table
    delta_h, delta_s = nn_table["init"]

    # Check if sequence is all A/T or contains at least one G/C
    if Seq(seq).count("G") + Seq(seq).count("C") == 0:
        delta_h += nn_table["init_allA/T"][d_h]
        delta_s += nn_table["init_allA/T"][d_s]
    else:
        delta_h += nn_table["init_oneG/C"][d_h]
        delta_s += nn_table["init_oneG/C"][d_s]

    # Penalty if 5' end is T or 3' end is A
    if seq.startswith("T"):
        delta_h += nn_table["init_5T/A"][d_h]
        delta_s += nn_table["init_5T/A"][d_s]
    if seq.endswith("A"):
        delta_h += nn_table["init_5T/A"][d_h]
        delta_s += nn_table["init_5T/A"][d_s]

    # Terminal base pair corrections
    ends = seq[0] + seq[-1]
    AT = ends.count("A") + ends.count("T")
    GC = ends.count("G") + ends.count("C")

    delta_h += nn_table["init_A/T"][d_h] * AT + nn_table["init_G/C"][d_h] * GC
    delta_s += nn_table["init_A/T"][d_s] * AT + nn_table["init_G/C"][d_s] * GC

    # Symmetry correction if sequence is self-complementary
    if seq == c_seq[::-1]:
        delta_h += nn_table["sym"][d_h]
        delta_s += nn_table["sym"][d_s]

    return delta_h, delta_s

In [ ]:
def determine_shift(seq, c_seq):
    seq = seq.upper()
    c_seq = c_seq.upper()

    c_seq_comp = str(Seq(c_seq).complement())

    max_shift = len(c_seq_comp) - 1
    best_shift = None

    # Shift primer to the right (positive shift, template 5'-dangling)
    for shift in range(0, max(len(c_seq_comp), len(seq))):
        overlap_len = min(len(seq), len(c_seq_comp) - shift)
        if overlap_len <= 0:
            continue
        if seq[:overlap_len] == c_seq_comp[shift:shift + overlap_len]:
            return shift

    # Primer shifted left (negative shift)
    for shift in range(1, len(seq)):
        overlap_len = min(len(c_seq_comp), len(seq) - shift)
        if overlap_len <= 0:
            continue
        if seq[shift:shift + overlap_len] == c_seq_comp[:overlap_len]:
            return -shift

    raise ValueError("No alignment found between seq and c_seq.")

# 2: Calculation Functions

In [ ]:
def calculate_matching(seq, c_seq, nn_table):
    # Zipping (nearest-neighbor calculation)
    delta_h, delta_s = initialize_thermodynamics(nn_table, seq)
    for i in range(len(seq) - 1):
        neighbors = seq[i:i+2] + '/' + c_seq[i:i+2]
        dh_nn, ds_nn = nn_table.get(neighbors, nn_table.get(neighbors[::-1], (0, 0)))
        delta_h += dh_nn
        delta_s += ds_nn
    return delta_h, delta_s

In [ ]:
def calculate_internal_mismatch(seq, c_seq, nn_table, imm_table):
    delta_h, delta_s = initialize_thermodynamics(nn_table, seq)
    # Nearest-neighbor calculation (internal mismatch)
    for i in range(len(seq) - 1):
        neighbors = seq[i:i+2] + '/' + c_seq[i:i+2]
        if neighbors in imm_table:
            dh, ds = imm_table[neighbors]
        elif neighbors[::-1] in imm_table:
            dh, ds = imm_table[neighbors[::-1]]
        elif neighbors in nn_table:
            dh, ds = nn_table[neighbors]
        elif neighbors[::-1] in nn_table:
            dh, ds = nn_table[neighbors[::-1]]
        else:
            raise ValueError(f"No data found for '{neighbors}'.")

        delta_h += dh
        delta_s += ds

    return delta_h, delta_s

In [ ]:
def calculate_terminal_mismatch(seq, c_seq, nn_table, tmm_table):
    delta_h, delta_s = initialize_thermodynamics(nn_table, seq)

    # **Apply Terminal Mismatch (TMM) corrections**
    # 5'-end terminal mismatch
    term5 = seq[:2] + '/' + c_seq[:2]
    dh_tmm, ds_tmm = tmm_table.get(term5, tmm_table.get(term5[::-1], (0, 0)))
    delta_h += dh_tmm
    delta_s += ds_tmm

    # 3'-end terminal mismatch
    term3 = seq[-2:] + '/' + c_seq[-2:]
    dh_tmm, ds_tmm = tmm_table.get(term3, tmm_table.get(term3[::-1], (0, 0)))
    delta_h += dh_tmm
    delta_s += ds_tmm

    # **Nearest-neighbor calculations (excluding terminal ends)**
    for i in range(len(seq) - 1):  # Fixed range
        neighbors = seq[i:i+2] + '/' + c_seq[i:i+2]
        dh, ds = nn_table.get(neighbors, nn_table.get(neighbors[::-1], (0, 0)))
        delta_h += dh
        delta_s += ds

    return delta_h, delta_s

In [ ]:
def calculate_dangling_end(seq, c_seq, nn_table, de_table, strict):
    delta_h, delta_s = initialize_thermodynamics(nn_table, seq)
    shift = determine_shift(seq, c_seq)
    tmp_seq, tmp_cseq = seq, c_seq

    # Step 3: Align sequences based on determined shift
    if shift > 0:
        tmp_seq = "." * shift + seq
    elif shift < 0:
        tmp_cseq = "." * abs(shift) + c_seq

        # Step 3: Pad shorter sequence with dots
    if len(tmp_seq) > len(tmp_cseq):
        tmp_cseq += "." * (len(tmp_seq) - len(tmp_cseq))
    elif len(tmp_cseq) > len(tmp_seq):
        tmp_seq += "." * (len(tmp_cseq) - len(tmp_seq))

    # Step 4: Apply dangling-end corrections
    which_end = None
    if tmp_seq.startswith(".") or tmp_cseq.startswith("."):
        left_de = tmp_seq[:2] + "/" + tmp_cseq[:2]
        if left_de in de_table:
            dh, ds = de_table[left_de]
            delta_h += dh
            delta_s += ds
            which_end = "5'-dangling"
        tmp_seq, tmp_cseq = tmp_seq[1:], tmp_cseq[1:]

    if tmp_seq.endswith(".") or tmp_cseq.endswith("."):
        right_de = tmp_cseq[-2:][::-1] + "/" + tmp_seq[-2:][::-1]
        if right_de in de_table:
            dh, ds = de_table[right_de]
            delta_h += dh
            delta_s += ds
            which_end = "3'-dangling"
        tmp_seq, tmp_cseq = tmp_seq[:-1], tmp_cseq[:-1]

    # Step 4: Nearest-neighbor calculations
    length_matched = min(len(tmp_seq), len(tmp_cseq))

    for i in range(length_matched - 1):
        neighbors = tmp_seq[i : i+2] + "/" + tmp_cseq[i : i+2]
        if neighbors in nn_table:
            dh, ds = nn_table[neighbors]
        elif neighbors[::-1] in nn_table:
            dh, ds = nn_table[neighbors[::-1]]
        else:
            if strict:
                raise ValueError(f"Missing data for nearest neighbors '{neighbors}'")
            dh, ds = 0.0, 0.0
        delta_h += dh
        delta_s += ds

    return delta_h, delta_s

In [ ]:
def calculate_duplex_thermodynamics(seq, c_seq=None, temperature=310.15,
                                    nn_table=None, imm_table=None, tmm_table=None,
                                    de_dna_table=None, de_rna_table=None,
                                    calc_type="Matching", strict=True):
    if not nn_table:
        raise ValueError("Error: Nearest-neighbor table (nn_table) is required.")

    seq = seq.upper()
    if calc_type == "Matching":
        if not c_seq:
            c_seq = str(Seq(seq).complement())
            print(f" Auto-generated complementary sequence: {c_seq}")
    elif not c_seq:
        raise ValueError("Complementary sequence (c_seq) is required.")

    if calc_type == "Matching":
        delta_h, delta_s = calculate_matching(seq, c_seq, nn_table)
    elif calc_type == "Internal Mismatch" and imm_table:
        delta_h, delta_s = calculate_internal_mismatch(seq, c_seq, nn_table, imm_table)
    elif calc_type == "Terminal Mismatch" and tmm_table:
        delta_h, delta_s = calculate_terminal_mismatch(seq, c_seq, nn_table, tmm_table)
    elif calc_type == "Dangling-End DNA" and de_dna_table:
        delta_h, delta_s = calculate_dangling_end(seq, c_seq, nn_table, de_dna_table, strict)
    elif calc_type == "Dangling-End RNA" and de_rna_table:
        delta_h, delta_s = calculate_dangling_end(seq, c_seq, nn_table, de_rna_table, strict)
    else:
        raise ValueError("Invalid calculation type or missing required table.")

    delta_g = delta_h - (temperature * (delta_s / 1000))
    return (delta_h, delta_s, delta_g)

# 3: User Input & Selection

In [ ]:
def select_nn_table(choice):

    # Nearest-Neighbor (NN) models for DNA, RNA, and RNA-DNA duplexes
    nn_models = {
        "DNA": {
            "1": ("DNA_NN1", "Breslauer 1986", DNA_NN1),
            "2": ("DNA_NN2", "Sugimoto 1996", DNA_NN2),
            "3": ("DNA_NN3", "SantaLucia 1997", DNA_NN3),
            "4": ("DNA_NN4", "SantaLucia & Hicks 2004", DNA_NN4),
            "5": ("DNA_NN5", "SantaLucia 1996", DNA_NN5),
        },
        "RNA": {
            "1": ("RNA_NN1", "Freier 1986", RNA_NN1),
            "2": ("RNA_NN2", "Xia 1998", RNA_NN2),
            "3": ("RNA_NN3", "Chen 2012", RNA_NN3),
        },
        "RNA_DNA": {
            "1": ("R_DNA_NN1", "Sugimoto 1995", R_DNA_NN1),
        }
    }

    # Fixed tables for choices 4-7
    fixed_tables = {
        "4": ("DNA_IMM1", "Allawi 1997", DNA_IMM1),  # Internal Mismatch
        "5": ("DNA_TMM1", "SantaLucia 2001", DNA_TMM1),  # Terminal Mismatch
        "6": ("DNA_DE1", "Bommarito 2000", DNA_DE1),  # DNA Dangling-End
        "7": ("RNA_DE1", "Turner 2010", RNA_DE1),  # RNA Dangling-End
    }

    # Mapping choices to NN models
    mapping = {
        "1": ("DNA", None),  # DNA Matching Duplex
        "2": ("RNA", None),  # RNA Matching Duplex
        "3": ("RNA_DNA", None),  # RNA-DNA Hybrid Duplex
        "4": ("DNA", fixed_tables["4"]),  # Internal Mismatch Duplex
        "5": ("DNA", fixed_tables["5"]),  # Terminal Mismatch Duplex
        "6": ("DNA", fixed_tables["6"]),  # DNA Dangling-End Duplex
        "7": ("RNA", fixed_tables["7"]),  # RNA Dangling-End Duplex
    }

    # Get the correct NN models and fixed table
    if choice in mapping:
        model_key, fixed_table = mapping[choice]
        return {"nn_models": nn_models[model_key], "fixed_table": fixed_table}

    return {}  # Invalid choice

In [ ]:
def get_calculation_type(choice):
    return ("Matching" if choice in ["1", "2", "3"] else
            "Internal Mismatch" if choice == "4" else
            "Terminal Mismatch" if choice == "5" else
            "Dangling-End DNA" if choice == "6" else
            "Dangling-End RNA" if choice == "7" else None)

In [ ]:
#  User Input Handling with Temperature Input
def get_user_sequences(calc_type):
    sequences = []
    while True:
        seq = input("Enter sequence (5'→3') or press Enter to finish: ").strip().upper()
        if not seq:
            break

        # Automatically generate complement for matching duplexes
        if calc_type == "Matching":
            c_seq = str(Seq(seq).complement())  # Direct complement, no reverse
            print(f" Automatically generated complementary sequence: {c_seq}")

        else:  # For mismatches, dangling ends, etc., ask user for complementary sequence
            c_seq = input(f"Enter complementary sequence (3'→5') for {seq}: ").strip().upper()

        sequences.append((seq, c_seq))

    # Ask user for temperature input (default: 298.15 K)
    temperature = input("\nEnter temperature in Kelvin (default: 298.15 K): ").strip()
    temperature = float(temperature) if temperature else 298.15  # Default to 298.15 K if user doesn't enter anything

    return sequences, temperature

# 4: Processing & Execution

In [ ]:
def process_sequence(seq, c_seq, temperature, calc_type, nn_table, fixed_table, fixed_table_name):

    if calc_type == "Matching":
        return calculate_duplex_thermodynamics(seq, c_seq, temperature, nn_table, calc_type="Matching")
    elif calc_type == "Internal Mismatch" and fixed_table:
        return calculate_duplex_thermodynamics(seq, c_seq, temperature, nn_table, imm_table=fixed_table, calc_type="Internal Mismatch")
    elif calc_type == "Terminal Mismatch" and fixed_table:
        return calculate_duplex_thermodynamics(seq, c_seq, temperature, nn_table, tmm_table=fixed_table, calc_type="Terminal Mismatch")
    elif calc_type == "Dangling-End DNA" and fixed_table:
        return calculate_duplex_thermodynamics(seq, c_seq, temperature, nn_table, de_dna_table=fixed_table, calc_type="Dangling-End DNA")
    elif calc_type == "Dangling-End RNA" and fixed_table:
        return calculate_duplex_thermodynamics(seq, c_seq, temperature, nn_table, de_rna_table=fixed_table, calc_type="Dangling-End RNA")
    else:
        print("Error: Invalid selection or missing correction table.")
        return None, None, None

In [ ]:
def process_and_print_results(sequences, nn_models, temperature, calc_type, fixed_table, fixed_table_name, choice):
    results_summary = []
    for seq, c_seq in sequences:
        for _, (model_name, reference_year, nn_table) in nn_models.items():
            dH, dS, dG = process_sequence(seq, c_seq, temperature, calc_type, nn_table, fixed_table, fixed_table_name)
            if dH is not None:
                results_entry = {
                    "seq": seq,
                    "c_seq": c_seq,
                    "ΔH (kcal/mol)": round(dH, 2),
                    "ΔS (cal/mol·K)": round(dS, 2),
                    "ΔG (kcal/mol)": round(dG, 2),
                    "Model": model_name,
                    "Year": reference_year,
                    "Correction": fixed_table_name if choice in ["4", "5", "6", "7"] else None,
                }
                results_summary.append(results_entry)

    summary_df = pd.DataFrame(results_summary)
    summary_df = summary_df[["seq", "c_seq", "ΔH (kcal/mol)", "ΔS (cal/mol·K)", "ΔG (kcal/mol)", "Model", "Year", "Correction"]]

    print("\n **Final Results Summary:**")
    display(summary_df)
    print("\n Calculation Complete!\n")

In [ ]:
def run_thermodynamics_calculator():
    print("\n Welcome to the Duplex Thermodynamics Calculator!\n")

    while True:
        print("\n Select Duplex Calculation Type:")
        print("1 - DNA Matching Duplex")
        print("2 - RNA Matching Duplex")
        print("3 - RNA-DNA Hybrid Duplex")
        print("4 - DNA Internal Mismatch Duplex (DNA_IMM1)")
        print("5 - DNA Terminal Mismatch Duplex (DNA_TMM1)")
        print("6 - DNA Dangling-End Duplex (DNA_DE1)")
        print("7 - RNA Dangling-End Duplex (RNA_DE1)")
        print("8 - Exit")

        choice = input("\nEnter your choice: ").strip()
        if choice == "8":
            print(" Exiting... Thank you!")
            break

        nn_options = select_nn_table(choice)
        if not nn_options:
            print(" Invalid choice. Please select again.")
            continue

        calc_type = get_calculation_type(choice)
        sequences, temperature = get_user_sequences(calc_type=calc_type)

        nn_models = nn_options.get("nn_models", {})
        fixed_table_entry = nn_options.get("fixed_table", None)
        fixed_table_name, fixed_table = (fixed_table_entry[0], fixed_table_entry[2]) if fixed_table_entry else (None, None)

        process_and_print_results(sequences, nn_models, temperature, calc_type, fixed_table, fixed_table_name, choice)

# 5: Execution & Main Call

In [ ]:
# Run the calculator
if __name__ == "__main__":
    run_thermodynamics_calculator()


🚀 Welcome to the Duplex Thermodynamics Calculator!


📌 Select Duplex Calculation Type:
1 - DNA Matching Duplex
2 - RNA Matching Duplex
3 - RNA-DNA Hybrid Duplex
4 - DNA Internal Mismatch Duplex (DNA_IMM1)
5 - DNA Terminal Mismatch Duplex (DNA_TMM1)
6 - DNA Dangling-End Duplex (DNA_DE1)
7 - RNA Dangling-End Duplex (RNA_DE1)
8 - Exit

Enter your choice: 1
Enter sequence (5'→3') or press Enter to finish: CATTTAGATGACG
✅ Automatically generated complementary sequence: GTAAATCTACTGC
Enter sequence (5'→3') or press Enter to finish: 

Enter temperature in Kelvin (default: 298.15 K): 

📌 **Final Results Summary:**


,seq,c_seq,ΔH (kcal/mol),ΔS (cal/mol·K),ΔG (kcal/mol),Model,Year,Correction
0,CATTTAGATGACG,GTAAATCTACTGC,-90.4,-248.2,-16.40,DNA_NN1,Breslauer 1986,None
1,CATTTAGATGACG,GTAAATCTACTGC,-95.0,-261.5,-17.03,DNA_NN2,Sugimoto 1996,None
2,CATTTAGATGACG,GTAAATCTACTGC,-97.4,-272.5,-16.15,DNA_NN3,SantaLucia 1997,None
3,CATTTAGATGACG,GTAAATCTACTGC,-96.8,-270.8,-16.06,DNA_NN4,SantaLucia & Hicks 2004,None
4,CATTTAGATGACG,GTAAATCTACTGC,-91.1,-253.0,-15.67,DNA_NN5,SantaLucia 1996,None



✅ Calculation Complete!


📌 Select Duplex Calculation Type:
1 - DNA Matching Duplex
2 - RNA Matching Duplex
3 - RNA-DNA Hybrid Duplex
4 - DNA Internal Mismatch Duplex (DNA_IMM1)
5 - DNA Terminal Mismatch Duplex (DNA_TMM1)
6 - DNA Dangling-End Duplex (DNA_DE1)
7 - RNA Dangling-End Duplex (RNA_DE1)
8 - Exit

Enter your choice: 8
✅ Exiting... Thank you!
